In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bakery-sales/Bakery Sales.csv
/kaggle/input/bakery-sales/Bakery price.csv


# Market Basket Analysis - Python

In [2]:
import pandas as pd
import seaborn as sb 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df = pd.read_csv("/kaggle/input/bakery-sales/Bakery Sales.csv")
df.head()

,datetime,day of week,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,...,gateau chocolat,pandoro,cheese cake,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies
0,2019-07-11 15:35,Thur,23800.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,2019-07-11 16:10,Thur,15800.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2019-07-12 11:49,Fri,58000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-07-13 13:19,Sat,14800.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,2019-07-13 13:22,Sat,15600.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.drop(columns=['place'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            2421 non-null   object 
 1   day of week         2420 non-null   object 
 2   total               2420 non-null   float64
 3   angbutter           1973 non-null   float64
 4   plain bread         857 non-null    float64
 5   jam                 220 non-null    float64
 6   americano           412 non-null    float64
 7   croissant           747 non-null    float64
 8   caffe latte         193 non-null    float64
 9   tiramisu croissant  779 non-null    float64
 10  cacao deep          323 non-null    float64
 11  pain au chocolat    587 non-null    float64
 12  almond croissant    202 non-null    float64
 13  croque monsieur     0 non-null      float64
 14  mad garlic          0 non-null      float64
 15  milk tea            137 non-null    float64
 16  gateau

In [6]:
df_2 = df.fillna(0)
df_2.head()

,datetime,day of week,total,angbutter,plain bread,jam,americano,croissant,caffe latte,tiramisu croissant,...,gateau chocolat,pandoro,cheese cake,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies
0,2019-07-11 15:35,Thur,23800.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2019-07-11 16:10,Thur,15800.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-12 11:49,Fri,58000.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-07-13 13:19,Sat,14800.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2019-07-13 13:22,Sat,15600.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_2["datetime"] = pd.to_datetime(df["datetime"])
df_2["month"] = df_2["datetime"].dt.month
df_2["id_transaction"] = df_2.reset_index()["index"] + 1
df_2 = df_2[['id_transaction', 'datetime', 'month', 'day of week', 'total', 'angbutter', 'plain bread', 'jam',
       'americano', 'croissant', 'caffe latte', 'tiramisu croissant',
       'cacao deep', 'pain au chocolat', 'almond croissant', 'croque monsieur',
       'mad garlic', 'milk tea', 'gateau chocolat', 'pandoro', 'cheese cake',
       'lemon ade', 'orange pound', 'wiener', 'vanila latte', 'berry ade',
       'tiramisu', 'merinque cookies']]
df_2.head()

,id_transaction,datetime,month,day of week,total,angbutter,plain bread,jam,americano,croissant,...,gateau chocolat,pandoro,cheese cake,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies
0,1,2019-07-11 15:35:00,7.0,Thur,23800.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,2019-07-11 16:10:00,7.0,Thur,15800.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,2019-07-12 11:49:00,7.0,Fri,58000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2019-07-13 13:19:00,7.0,Sat,14800.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,5,2019-07-13 13:22:00,7.0,Sat,15600.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def encode(item_freq):
    res = 0
    if item_freq > 0:
        res = 1
    return res
    
basket_input = df_2[['angbutter', 'plain bread', 'jam',
       'americano', 'croissant', 'caffe latte', 'tiramisu croissant',
       'cacao deep', 'pain au chocolat', 'almond croissant', 'croque monsieur',
       'mad garlic', 'milk tea', 'gateau chocolat', 'pandoro', 'cheese cake',
       'lemon ade', 'orange pound', 'wiener', 'vanila latte', 'berry ade',
       'tiramisu', 'merinque cookies']].applymap(encode)

In [9]:
basket_input.head()

,angbutter,plain bread,jam,americano,croissant,caffe latte,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,...,gateau chocolat,pandoro,cheese cake,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies
0,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
frequent_itemsets = apriori(basket_input, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift")

rules

/opt/conda/lib/python3.7/site-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(plain bread),(angbutter),0.322909,0.743406,0.244160,0.756126,1.017110,0.004107,1.052157
1,(angbutter),(plain bread),0.743406,0.322909,0.244160,0.328434,1.017110,0.004107,1.008227
2,(jam),(angbutter),0.082894,0.743406,0.055765,0.672727,0.904926,-0.005859,0.784037
3,(angbutter),(jam),0.743406,0.082894,0.055765,0.075013,0.904926,-0.005859,0.991480
4,(americano),(angbutter),0.155237,0.743406,0.122833,0.791262,1.064374,0.007429,1.229263
...,...,...,...,...,...,...,...,...,...
65,"(croissant, angbutter)",(pain au chocolat),0.210249,0.221176,0.061417,0.292115,1.320737,0.014915,1.100213
66,"(pain au chocolat, angbutter)",(croissant),0.165787,0.281462,0.061417,0.370455,1.316180,0.014754,1.141360
67,(croissant),"(pain au chocolat, angbutter)",0.281462,0.165787,0.061417,0.218206,1.316180,0.014754,1.067049
68,(pain au chocolat),"(croissant, angbutter)",0.221176,0.210249,0.061417,0.277683,1.320737,0.014915,1.093359
